In [ ]:
!pip install tensorflow
!pip install pillow
!pip install matplotlib

In [ ]:
import os
import PIL
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D ,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img

img_dir = ".."
for expression in os.listdir(img_dir):
    print(expression, "folder contians\t\t", len(os.listdir(img_dir + expression)), 'images')

BATCH_SIZE = 64
IMAGE_SIZE = 150
input_shape = (150,150,1)

Data_gen = ImageDataGenerator(rescale = 1.0/255,validation_split = 0.2)


train_gen = Data_gen.flow_from_directory(img_dir,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size = BATCH_SIZE,
                                        color_mode= "grayscale",
                                        shuffle=True,
                                        class_mode = "binary",
                                        subset = "training")

valid_gen = Data_gen.flow_from_directory(img_dir,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size = BATCH_SIZE,
                                        color_mode= "grayscale",
                                        shuffle=True,
                                        class_mode = "binary",
                                        subset = "validation")

labels = train_gen.class_indices

In [ ]:
images, labels = train_gen.next()

fig, axes = plt.subplots(5, 5, figsize=(30, 10))
axes = axes.ravel()
for i in np.arange(0, 25):
    axes[i].imshow(images[i], cmap='gray')
    axes[i].set_title(labels[i])
    axes[i].axis('off')
plt.subplots_adjust(wspace=0.5)
plt.show()

In [ ]:
image = images[0]

# Create a new figure
fig, ax = plt.subplots()

# Plot the image
ax.imshow(image, cmap="gray")
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xticks([])
ax.set_yticks([])
fig.savefig('0.jpg', dpi=300)
ax.grid(color='w', linestyle='-', linewidth=1)
# Add a grid of all pixels on top of the image
#nrows, ncols = image.shape[:2]
#row, col = np.meshgrid(np.arange(nrows), np.arange(ncols), indexing='ij')
#ax.scatter(col, row, c=image.reshape(-1, 3), s=1, alpha=0.1, cmap='gray')
ax.set_xticks(np.arange(-0.5, 150, 1))
ax.set_yticks(np.arange(-0.5, 150, 1))
ax.set_xticklabels([])
ax.set_yticklabels([])
fig.savefig('1.jpg', dpi=300)
# Show the plot
plt.show()

In [ ]:
model = Sequential([
    Conv2D(32,(3,3),activation="relu",input_shape=input_shape),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(128,(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(512,activation="relu"),
    Dropout(0.2),
    Dense(1,activation="sigmoid")])

In [ ]:
model.compile(optimizer = tensorflow.keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights = True)
checkpoint = ModelCheckpoint("Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
history = model.fit(train_gen,verbose=1,callbacks = [early_stopping,checkpoint],epochs=20,validation_data=valid_gen)

In [ ]:
model.load_weights('Model.h5')
score = model.evaluate(valid_gen ,steps =( valid_gen.n // 64) + 1)
print('\n', 'Validation accuracy:', score[1])

In [ ]:
weights = model.layers[2].get_weights()[0]
fig, ax = plt.subplots()
ax.imshow(np.squeeze(weights[:, :, 0, 9]), cmap='gray')
ax.axis('off')
fig.savefig('f210.jpg', dpi=300)
plt.show()

In [ ]:
weights = model.layers[0].get_weights()[0]

# Reshape each filter to 28x28 and display as gray-scale image
fig, axes = plt.subplots(nrows=4, ncols=8, figsize=(10, 5))
for i in range(32):
    row = i // 8
    col = i % 8
    ax = axes[row, col]
    ax.imshow(np.squeeze(weights[:, :, 0, i]), cmap='gray')
    ax.axis('off')
plt.show()

In [ ]:
weights = model.layers[2].get_weights()[0]

# Reshape each filter to 28x28 and display as gray-scale image
fig, axes = plt.subplots(nrows=8, ncols=8, figsize=(10, 5))
for i in range(64):
    row = i // 8
    col = i % 8
    ax = axes[row, col]
    ax.imshow(np.squeeze(weights[:, :, 0, i]), cmap='gray')
    ax.axis('off')
plt.show()

In [ ]:
weights = model.layers[4].get_weights()[0]

# Reshape each filter to 28x28 and display as gray-scale image
fig, axes = plt.subplots(nrows=16, ncols=8, figsize=(10, 5))
for i in range(128):
    row = i // 8
    col = i % 8
    ax = axes[row, col]
    ax.imshow(np.squeeze(weights[:, :, 0, i]), cmap='gray')
    ax.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Model

# Choose the layer to visualize
layer_name = 'conv2d_3'

# Create a new model that outputs the intermediate activations of the chosen layer
activation_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

# Load an example image and preprocess it
image = images[0]#plt.imread('X')
image = np.expand_dims(image, axis=0)
#image = image / 255.0

# Predict the intermediate activations for the example image
activations = activation_model.predict(image)

# Plot the feature maps
fig, axes = plt.subplots(nrows=4, ncols=8, figsize=(14, 7))
for i in range(activations.shape[-1]):
    row = i // 8
    col = i % 8
    ax = axes[row, col]
    ax.imshow(activations[0, :, :, i], cmap='gray')
    ax.axis('off')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Model

# Choose the layer to visualize
layer_name = 'conv2d_20'

# Create a new model that outputs the intermediate activations of the chosen layer
activation_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

# Load an example image and preprocess it
image = images[25]#plt.imread('X')
image = np.expand_dims(image, axis=0)
#image = image / 255.0
conv1_output = activation_model.predict(image)[0,:,:,9]

# Visualize the output
fig, ax = plt.subplots()
ax.imshow(conv1_output, cmap='gray')
ax.axis('off')
fig.savefig('fmx4.jpg', dpi=300)

plt.show()

In [ ]:
# Load the trained model
#model = model.load_weights('Model.h5')

# Load the CIFAR-10 dataset
valid_images, valid_labels = valid_gen.next()

# Preprocess the data
#X_test = X_test / 255.0

# Make predictions on the test set
y_pred = np.round(model.predict(valid_images))

# Select 10 random examples
#idx = np.random.choice(X_test.shape[0], size=10, replace=False)
correct_color = 'darkgreen'
incorrect_color = 'red'
# Plot the examples with their true label and predicted label
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(10, 5),gridspec_kw={'hspace': 0.75, 'wspace': 0})
for i, ax in enumerate(axes.flat):
    ax.imshow(valid_images[i], cmap="gray")

    color = correct_color if valid_labels[i] == y_pred[i][0] else incorrect_color
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title("True: %s \nPredicted: %s" % (valid_labels[i], y_pred[i][0]), color=color)

fig.savefig('tf.jpg', dpi=300)
plt.show()